Attempting to load the occurrence dataframe and make it smaller by retaining just the necessary columns.

In [1]:
import os
import pandas as pd
import sys
from collections import Counter

from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn

In [2]:
home_dir = os.path.dirname(os.getcwd())
data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/"

family_name = "Sesiidae"

In [3]:
# Remove the extra rows and compare the size
fields_to_keep = [
    "id",
    "decimalLatitude",
    "decimalLongitude",
    "order",
    "family",
    "genus",
    "species",
    "acceptedScientificName",
    "year",
    "month",
    "day",
    "datasetName",
    "taxonID",
    "acceptedTaxonKey",
    "lifeStage",
    "basisOfRecord",
]

In [4]:
with DwCAReader(os.path.join(data_dir,"dwca_files",family_name+".zip")) as dwca:
    occ_df_usecols = dwca.pd_read("occurrence.txt", 
                                  parse_dates=True,
                                  on_bad_lines="skip",
                                  usecols = fields_to_keep)


/bask/projects/v/vjgo8416-amber/conda_envs/gbif-species-trainer-AMI-fork/lib/python3.8/site-packages/dwca/read.py:203: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(self.absolute_temporary_path(relative_path), **kwargs)


In [5]:
sys.getsizeof(occ_df_usecols)/1024/1024

85.77149105072021

In [8]:
occ_df_usecols_trunk = occ_df_usecols[fields_to_keep]

In [9]:
sys.getsizeof(occ_df_usecols_trunk)/1024/1024

78.22808361053467

In [10]:
occ_df_usecols.columns.difference(occ_df_usecols_trunk.columns)

Index(['geodeticDatum'], dtype='object')